In [ ]:
import os
import re
import cv2
import numpy as np
import tensorflow as tf
from collections import deque
import tensorflow as tf
from IPython.display import Video

from tensorflow.keras.models import load_model
import pickle



In [ ]:
# Load your trained model.
model = load_model('/content/Final_LRCN.keras')



In [ ]:
with open("/content/mean.pkl", "rb") as f:
    mean = pickle.load(f)

In [ ]:
with open("/content/std_array.pkl", "rb") as f:
    std = pickle.load(f)

In [ ]:
with open("/content/class_list1.pkl", "rb") as f:
    encoder1 = pickle.load(f)

In [ ]:
youtube_url = 'https://www.youtube.com/watch?v=hMTtOjcOFbQ'

In [ ]:
# Define parameters.
SEQUENCE_LENGTH = 25
IMAGE_WIDTH = 112
IMAGE_HEIGHT = 112



In [ ]:
!pip install yt_dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 36.2 MB/s eta 0:00:00


In [ ]:
# CLASS_LIST = ['PushUps', 'PlayingGuitar', 'HighJump', 'RopeClimbing', 'PullUps', 'PoleVault', 'Kayaking', 'Mixing']
# CLASS_LIST = ['BaseballPitch', 'BenchPress', 'Billiards', 'BreastStroke', 'PlayingPiano', 'Drumming', 'Fencing']
# my_classes = ['JugglingBalls', 'Swing', 'RockClimbingIndoor','HighJump', 'Skijet', 'SkateBoarding', 'MilitaryParade']

##Downloading the youtube video


In [ ]:
import os
import re
import yt_dlp

def download_youtube_video(youtube_video_url, output_directory):
    os.makedirs(output_directory, exist_ok=True)

    # Get the next available sequential filename
    existing_files = os.listdir(output_directory)
    video_files = sorted([f for f in existing_files if f.startswith("vid") and f.endswith((".mp4", ".mkv", ".webm"))])

    if video_files:
        # Extract the largest sequence number
        last_seq = max(int(re.search(r'\d+', f).group()) for f in video_files)
        new_seq = last_seq + 1
    else:
        new_seq = 1

    # Format the new filename as vidXX (e.g., vid01, vid02)
    new_filename = f"vid{new_seq:02d}"

    # Configure yt-dlp options
    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',
        'outtmpl': f'{output_directory}/{new_filename}.%(ext)s',
    }

    # Download the video
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(youtube_video_url, download=True)
        ext = info_dict.get('ext', 'mp4')  # Get extension, default to mp4 if not available

    # Return the file path and new filename
    file_path = os.path.join(output_directory, f"{new_filename}.{ext}")
    return file_path, new_filename


In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
print(encoder1.classes_)

['BaseballPitch' 'BreastStroke' 'Drumming' 'MilitaryParade' 'PlayingPiano'
 'SkateBoarding' 'Skijet']


Prediction on video

In [ ]:
# Function to perform frame-by-frame action recognition on the video.
def predict_on_video(input_video_path, output_video_path, model, SEQUENCE_LENGTH):
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print("Error: Could not open video file.")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc('M','P','4','V')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    frames_buffer = deque(maxlen=SEQUENCE_LENGTH)
    predicted_class_name = ''

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break



    # while True:
    #     ret, frame = cap.read()
    #     if not ret:
    #         break

        # Preprocess the frame.
        frame_resized = cv2.resize(frame, (IMAGE_WIDTH, IMAGE_HEIGHT))
        frame_norm = frame_resized / 255.0
        frames_buffer.append(frame_norm)

        # Once we have a full sequence, predict.
        if len(frames_buffer) == SEQUENCE_LENGTH:
            # sequence = np.array(frames_buffer)  # shape: (SEQUENCE_LENGTH, 112, 112, 3)

            frames_buffer -= mean
            frames_buffer /= std
            sequence = np.expand_dims(frames_buffer, axis=0)  # shape: (1, SEQUENCE_LENGTH, 112, 112, 3)
            prediction = model.predict(sequence)[0]
            predicted_label = np.argmax(prediction)
            # predicted_class_name = CLASS_LIST[predicted_label]
            predicted_class_name = encoder1.inverse_transform([predicted_label])[0]

            # Annotate the current frame with the predicted class.
            cv2.putText(frame, f'Predicted: {predicted_class_name}', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            cv2_imshow(frame)
            print(f'Action Predicted: {predicted_class_name}\n')

        out.write(frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()




In [ ]:
# Function to perform a single action prediction (using SEQUENCE_LENGTH frames sampled from the video).
def predict_single_action(video_file_path, SEQUENCE_LENGTH):
    cap = cv2.VideoCapture(video_file_path)
    if not cap.isOpened():
        print("Error: Could not open video file.")
        return

    frames_list = []
    video_frames_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    skip_frames_window = max(int(video_frames_count / SEQUENCE_LENGTH), 1)

    for frame_counter in range(SEQUENCE_LENGTH):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        ret, frame = cap.read()
        if not ret:
            break
        resized_frame = cv2.resize(frame, (IMAGE_WIDTH, IMAGE_HEIGHT))
        normalized_frame = resized_frame / 255.0
        frames_list.append(normalized_frame)



    if len(frames_list) != SEQUENCE_LENGTH:
        print("Warning: Not enough frames for a full sequence.")
        return

    frames_list -= mean
    frames_list /= std
    sequence = np.expand_dims(np.array(frames_list), axis=0)
    prediction = model.predict(sequence)[0]
    predicted_index = np.argmax(prediction)
    # predicted_class_name = CLASS_LIST[predicted_index]
    # predicted_class_name = index_to_label[predicted_index]
    predicted_class_name = encoder1.inverse_transform([predicted_index])[0]
    confidence = prediction[predicted_index]
    print(f'Action Predicted: {predicted_class_name}\nConfidence: {confidence}')

    cap.release()

In [ ]:
test_videos_directory = 'test_videos'
os.makedirs(test_videos_directory, exist_ok=True)



In [ ]:
# Download a YouTube video. (You can change the URL as needed.)

input_video_path, safe_title = download_youtube_video(youtube_url, test_videos_directory)


[youtube] Extracting URL: https://www.youtube.com/watch?v=hMTtOjcOFbQ
[youtube] hMTtOjcOFbQ: Downloading webpage
[youtube] hMTtOjcOFbQ: Downloading tv client config
[youtube] hMTtOjcOFbQ: Downloading player 73381ccc-main
[youtube] hMTtOjcOFbQ: Downloading tv player API JSON
[youtube] hMTtOjcOFbQ: Downloading ios player API JSON
[youtube] hMTtOjcOFbQ: Downloading m3u8 information
[info] hMTtOjcOFbQ: Downloading 1 format(s): 303+251
[download] Destination: test_videos/vid01.f303.webm
[download] 100% of   14.72MiB in 00:00:00 at 32.65MiB/s  
[download] Destination: test_videos/vid01.f251.webm
[download] 100% of  382.48KiB in 00:00:00 at 5.37MiB/s   
[Merger] Merging formats into "test_videos/vid01.webm"
Deleting original file test_videos/vid01.f251.webm (pass -k to keep)
Deleting original file test_videos/vid01.f303.webm (pass -k to keep)


In [ ]:
def frames_extraction(video_path):
    frames_list = []
    video_reader = cv2.VideoCapture(video_path)
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    skip_frames_window = max(int(video_frames_count/25), 1)
    for frame_counter in range(25):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter)
        succes, frame = video_reader.read()
        if not succes:
            break
        frames_list.append(frame)
    video_reader.release()

    return frames_list

In [ ]:
if input_video_path is None:
    print("Error: Video download failed or file not found.")
else:
    print(f"Downloaded video path: {input_video_path}")
    print("Files in directory:", os.listdir(test_videos_directory))

    # Construct the output video path.
    output_video_path = os.path.join(test_videos_directory, f"{safe_title}-Output-SeqLen{SEQUENCE_LENGTH}.mp4")
    print("Output video path:", output_video_path)

    # Perform single action prediction on the downloaded video.
    predict_single_action(input_video_path, SEQUENCE_LENGTH)

Downloaded video path: test_videos/vid01.webm
Files in directory: ['vid01.webm']
Output video path: test_videos/vid01-Output-SeqLen25.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Action Predicted: Skijet
Confidence: 0.6569247841835022


In [ ]:
if input_video_path is None:
    print("Error: Video download failed or file not found.")
else:
    print(f"Downloaded video path: {input_video_path}")
    print("Files in directory:", os.listdir(test_videos_directory))

    # Construct the output video path.
    output_video_path = os.path.join(test_videos_directory, f"{safe_title}-Output-SeqLen{SEQUENCE_LENGTH}.mp4")
    print("Output video path:", output_video_path)

    # Run continuous prediction on the video.
    predict_on_video(input_video_path, output_video_path, model, SEQUENCE_LENGTH)

    # Display the output video in the notebook.
    from IPython.display import Video
    Video(output_video_path, embed=True)


    # # Perform single action prediction on the downloaded video.
    # predict_single_action(input_video_path, SEQUENCE_LENGTH)

for my rough work

In [ ]:
def Human_activity_prediction(video_path , mean , std):
    frames = frames_extraction(video_path)
    if len(frames) != 25:
        raise ValueError(f"Expected {25} frames but got {len(frames)} frames from the video.")
    preprocessed_frames = []
    for frame in frames:
        resized_frame = cv2.resize(frame, (112, 112))
        preprocessed_frames.append(resized_frame)

    preprocessed_frames= (np.array(preprocessed_frames)).astype(np.float32)
    preprocessed_frames -= mean
    preprocessed_frames /= std


    preprocessed_frames = np.expand_dims(preprocessed_frames, axis=0)

    predicted_probabilities = model.predict(preprocessed_frames)[0]
    print(predicted_probabilities)
    predicted_label = np.argmax(predicted_probabilities)
    predicted_class = encoder1.inverse_transform([predicted_label])[0]

    print(predicted_class)

In [ ]:
Human_activity_prediction(input_video_path,mean,std)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
[0.03632772 0.00226159 0.29508966 0.63694936 0.02332744 0.00348251
 0.00256167]
MilitaryParade
